# Analyzing Parade

As a board game enjoyer, I recently discovered an excellent card game called [Parade](https://boardgamegeek.com/boardgame/56692/parade). It is not that different from [Arboretum](https://boardgamegeek.com/boardgame/140934/arboretum), except it's more random so less cutthroat. Basically, you try to take as few cards as possible from the parade because cards score negative points. After playing a couple games, I came up with some hypothesis about the game's strategy and mechanics.

Unlike Arboretum, where the general strategy is to play small discard small. It is hard to come up with a strategy for Parade. It feels very random. I decided to simulate the game and see what I could come up with. I impelemented the simulator in Rust to gather some statistics using a 90%-there strategy. It's not perfect, but it's good enough to give some ideas about Parade's characteristics, especially just how much does the early game affects the game's outcome. I also wanted to optimize to avoid being forced to take cards from the parade, which is a bad experience. 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.ticker import PercentFormatter
import itertools

The simulator takes different parameters. players count, number of suits, number of ranks, number of iterations, and strategies. The strategy is a function that takes a player's hand and returns a card to add to the parade. The simulator runs the game for a number of iterations and outputs the score of the game. Then in pandas, I calculate the statistics for configuration.

In [2]:
def run(players=2, suits=6, ranks=11, iters=50000, strats=[1]):
    result = []
    for s in itertools.product(strats, repeat=players):
        strat = " ".join(list(map(str,s)))
        ! cargo run --release -- --players=$players --suits=$suits --ranks=$ranks --iters=$iters $strat
        df = pd.read_csv("output.csv", header=None)
        df = df.set_axis(["score "+str(i) for i in range(players)]+ ["forced take "+str(i) for i in range(players)], axis=1)
        means = df.mean()
        for i, m in enumerate(s): 
            means[str(i)] = m
        result.append(means)
    return pd.DataFrame(result)

I wrote 3 strategies:

1. Always use the first card in hand.
2. For each card in hand, find the lowest sum of ranks from the ejected cards.
3. On top of 2, voluntarily take rank 0-2 cards from the parade.

In [3]:
df = run(players=2, suits=6, ranks=11, iters=5000, strats=[0, 1, 2])
df['scorediff'] = df['score 0'] - df['score 1']
df['forcedtakediff'] = df['forced take 0'] - df['forced take 1']
df

   Compiling ryu v1.0.10
   Compiling itoa v1.0.2
   Compiling serde v1.0.137
   Compiling serde_json v1.0.81
   Compiling wasm-bindgen v0.2.80     ] 43/88: itoa, serde_json(build), ...
   Compiling getrandom v0.2.6         ] 44/88: itoa, serde_json(build), ...
   Compiling clap v3.1.18======>      ] 71/88: itoa, ryu, serde, getrandom 
   Compiling rand_core v0.6.3====>    ] 76/88: clap, ryu, serde, getrandom 
   Compiling rand_chacha v0.3.1==>    ] 78/88: clap, serde, rand_core, g...
   Compiling rand v0.8.5==========>   ] 80/88: clap, rand_chacha, serde    
   Compiling parade v0.1.0 (/workspaces/boardgames/parade)                 
   --> src/parade.rs:120:8
    |
120 |     fn commit_end_game(&mut self) {
    |        ^^^^^^^^^^^^^^^
    |
    = note: `#[warn(dead_code)]` on by default

   --> src/parade.rs:128:12
    |
128 |     pub fn final_score(&self) -> Vec<usize> {
    |            ^^^^^^^^^^^

   --> src/parade.rs:164:8
    |
164 | pub fn simulate(cfg: &Config, seed: usize) ->

,score 0,score 1,forced take 0,forced take 1,0,1,scorediff,forcedtakediff
0,81.6048,94.8418,3.0858,3.2364,0.0,0.0,-13.2370,-0.1506
1,157.0064,29.7596,7.1576,10.0456,0.0,1.0,127.2468,-2.8880
2,155.1308,30.5842,7.0462,9.7232,0.0,2.0,124.5466,-2.6770
3,23.8352,168.5916,9.8170,7.4356,1.0,0.0,-144.7564,2.3814
4,71.9792,84.6086,15.2754,15.5552,1.0,1.0,-12.6294,-0.2798
5,70.9816,85.0946,15.1768,15.3854,1.0,2.0,-14.1130,-0.2086
6,24.6676,167.4700,9.4420,7.3156,2.0,0.0,-142.8024,2.1264
7,72.4590,84.0288,15.0680,15.4582,2.0,1.0,-11.5698,-0.3902
8,71.6764,84.5416,15.0130,15.3230,2.0,2.0,-12.8652,-0.3100


In [4]:
df.pivot_table(index="0", columns="1", values='scorediff', fill_value=0)

1,0.0,1.0,2.0
0,,,
0.0,-13.2370,127.2468,124.5466
1.0,-144.7564,-12.6294,-14.1130
2.0,-142.8024,-11.5698,-12.8652


In [5]:
df.pivot_table(index="0", columns="1", values='forcedtakediff', fill_value=0)

1,0.0,1.0,2.0
0,,,
0.0,-0.1506,-2.8880,-2.6770
1.0,2.3814,-0.2798,-0.2086
2.0,2.1264,-0.3902,-0.3100


Next let's look at number of forced takes.

In [6]:
results = []
for i in range(4, 10):
    for j in range(5, 20):
        df = run(players=2, suits=i, ranks=j, iters=5000)
        df['score mean'] = df.loc[:, df.columns.str.startswith('score')].loc[0].mean()
        df['forced take mean'] = df.loc[:, df.columns.str.startswith('forced take')].loc[0].mean()
        df["suits"] = i
        df["ranks"] = j
        results.append(df)
df = pd.concat(results)
df

   --> src/parade.rs:120:8
    |
120 |     fn commit_end_game(&mut self) {
    |        ^^^^^^^^^^^^^^^
    |
    = note: `#[warn(dead_code)]` on by default

   --> src/parade.rs:128:12
    |
128 |     pub fn final_score(&self) -> Vec<usize> {
    |            ^^^^^^^^^^^

   --> src/parade.rs:164:8
    |
164 | pub fn simulate(cfg: &Config, seed: usize) -> (Parade, Vec<Stats>) {
    |        ^^^^^^^^

    Finished release [optimized] target(s) in 0.03s
     Running `target/release/parade --players=2 --suits=4 --ranks=5 --iters=5000 1 1`
   --> src/parade.rs:120:8
    |
120 |     fn commit_end_game(&mut self) {
    |        ^^^^^^^^^^^^^^^
    |
    = note: `#[warn(dead_code)]` on by default

   --> src/parade.rs:128:12
    |
128 |     pub fn final_score(&self) -> Vec<usize> {
    |            ^^^^^^^^^^^

   --> src/parade.rs:164:8
    |
164 | pub fn simulate(cfg: &Config, seed: usize) -> (Parade, Vec<Stats>) {
    |        ^^^^^^^^

    Finished release [optimized] target(s) in 0.04s


,score 0,score 1,forced take 0,forced take 1,0,1,score mean,forced take mean,suits,ranks
0,1.9248,2.1638,1.4354,1.3520,1.0,1.0,2.0443,1.3937,4,5
0,5.1610,6.2880,2.7030,2.6400,1.0,1.0,5.7245,2.6715,4,6
0,9.5206,12.0072,3.5808,3.8294,1.0,1.0,10.7639,3.7051,4,7
0,15.4716,19.3502,4.6370,4.8356,1.0,1.0,17.4109,4.7363,4,8
0,22.7042,28.2432,5.5924,5.8732,1.0,1.0,25.4737,5.7328,4,9
...,...,...,...,...,...,...,...,...,...,...
0,266.2198,279.6484,37.1844,37.2934,1.0,1.0,272.9341,37.2389,9,15
0,273.6316,314.1106,38.9180,39.2712,1.0,1.0,293.8711,39.0946,9,16
0,342.9668,361.7312,41.3544,41.7000,1.0,1.0,352.3490,41.5272,9,17
0,354.5326,400.8610,42.9150,43.1482,1.0,1.0,377.6968,43.0316,9,18


In [7]:
pd.pivot_table(df, index="suits", columns="ranks", values='forced take mean', fill_value=0)

ranks,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
suits,,,,,,,,,,,,,,,
4,1.3937,2.6715,3.7051,4.7363,5.7328,6.6863,7.5220,8.2940,9.0200,9.6886,10.3170,10.8866,11.4638,12.0163,12.5411
5,3.4853,4.7480,6.5679,7.6225,9.2888,10.2058,11.7039,12.4790,13.8229,14.4225,15.6158,16.1143,17.1225,17.6508,18.5414
6,4.8187,6.8283,8.7015,10.5030,12.1796,13.8364,15.4153,16.8259,18.1851,19.4250,20.6266,21.6870,22.7442,23.7022,24.5359
7,6.7894,8.8423,11.4850,13.3116,15.7737,17.4235,19.6746,21.1669,23.2415,24.5027,26.3872,27.4435,29.0822,30.0510,31.5419
8,7.9993,10.8489,13.5135,16.1096,18.5309,20.9440,23.2862,25.4768,27.6009,29.6213,31.4866,33.3068,34.9863,36.5348,38.0212
9,9.8985,12.7504,16.1637,18.7654,22.0142,24.4033,27.4777,29.7262,32.5959,34.6043,37.2389,39.0946,41.5272,43.0316,45.2487


In [8]:
pd.pivot_table(df, index="suits", columns="ranks", values='score mean', fill_value=0)

ranks,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
suits,,,,,,,,,,,,,,,
4,2.0443,5.7245,10.7639,17.4109,25.4737,34.7954,45.3743,57.1040,70.5069,84.7516,100.3951,117.5875,135.4955,155.2412,175.9935
5,13.3021,10.6557,27.2354,26.4609,47.5101,48.5490,73.3919,76.4655,105.5296,110.7920,143.8189,151.7058,188.6638,198.8190,239.5375
6,8.6008,15.6587,24.6091,35.3802,48.0318,62.0873,78.2939,95.8370,115.9037,137.5155,160.9433,186.5107,213.2233,242.1693,273.6661
7,20.8429,20.7053,42.5510,44.7295,72.1765,76.1612,109.0941,116.4246,154.3027,164.2444,207.5411,221.3110,269.5838,286.6028,339.6460
8,15.4655,25.8935,38.7534,53.8689,71.3187,90.8583,112.6421,136.9232,163.2014,192.5650,223.8025,257.8069,293.1283,331.6588,372.8415
9,28.5251,31.1371,57.9848,63.3492,96.9963,105.5200,145.8092,157.7437,204.3985,220.2339,272.9341,293.8711,352.3490,377.6968,441.9688


So if you really hate being forced to take a card from the parade, you can playing with 4 suits and 5 ranks.